#Trabalho de Conclusão de Curso

##Aplicação de modelos de processamento de linguagem natural em um catálogo de vulnerabilidades cibernéticas

###Identificação

**Aluno:**  José Caetano Beuker

**Curso:**  MBA em Data Science e Analytics

**IE:** Escola Superior de Agricultura Luiz de Queiroz - Universidade de São Paulo

### Importação de bibliotecas utilizadas

In [ ]:
# Para a manipulação de dataframes e análise de dados
import pandas as pd

# Biblioteca para trabalhar com arrays grandes e multidimensionais e também matrizes
import numpy as np

# Para visualizações
import seaborn as sns

# Para aprendizado de máquina
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression


# Para regex - regular expression
import re

# Para nuvem de palavras
from wordcloud import WordCloud

# Para visualização
import matplotlib.pyplot as plt

# Natural Language Toolkit - para PLN
import nltk 

# from nltk import ngrams

# Para remoção de stop words
from nltk.corpus import stopwords 

# Atualiza a lista de stop words
#nltk.download('stopwords')

# Cria objeto para remover stop words em Inglês
stop_words_en = stopwords.words('english')

from nltk.tokenize import word_tokenize 

#from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer

from nltk.stem import PorterStemmer

porter = PorterStemmer()
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = WordNetLemmatizer()


#print(stopwords.words('english'))

### Funções utilizadas

In [ ]:
# Criar objeto para remover caracteres especiais
remove_caracteres_especiais = re.compile('[/(){}\[\]\|@,;]')

# Objeto para manter apenas caracteres alfanuméricos
mantem_apenas_alfanumerico = re.compile('[^0-9a-z #+_]')

# Função para limpar os textos das colunas do dataframe
def limpa_texto(texto):
    # Converte o texto para letras minúsculas
    texto = texto.lower()

    # Subsitui caracteres por espaços 
    texto = remove_caracteres_especiais.sub(' ', texto)

    # Remote caracteres que não forem alfanuméricos
    texto = mantem_apenas_alfanumerico.sub('', texto)
    
    return texto 

# Função para tokenizar as palavras de uma coluna de um DataFrame
def tokeniza_palavras_coluna (dataframe, coluna):
    palavras = ' '.join([word for word in dataframe[coluna]])
    # Realiza a tokenização
    tokens = word_tokenize(palavras)
    return tokens


# Função para preparar exibição de gráfico de ocorrências de palavras em uma coluna de um DataFrame
def prepara_para_grafico (dataframe, coluna):
    # Realiza a tokenização
    tokens = tokeniza_palavras_coluna(dataframe, coluna)
    # Obtém a frequência de ocorrências do token
    frequencia = nltk.FreqDist(tokens)
    pd_frequencia = pd.DataFrame({"token": list(frequencia.keys()),
                                 "frequencia": list(frequencia.values())})
    return pd_frequencia

# Função para exibir gráfico de Pareto de ocorrências de palavras em uma coluna de um DataFrame
def exibe_pareto (dados, coluna, quantidade_colunas, cor, titulo):
    plt.figure(figsize=(10, 6))
    x = sns.barplot(data = dados.nlargest(columns = coluna, n = quantidade_colunas), x = "token", y = "frequencia", color = cor)
    x.set(ylabel = "Quantidade", xlabel = "Tokens", title = titulo)
    plt.show()

# Função para remover stop words
def remove_stop_words(word_list, stop_word_list):
    tokens = word_tokenize(word_list)
    lista_sem_stop_words = ' '.join([word for word in tokens 
                                     if word not in stop_word_list])
    return lista_sem_stop_words

# Função de Stemming
def stemming (texto):
    stemmings = [porter.stem(word) for word in word_tokenize(texto)]
    return ' '.join(stemmings)

# Função de Lematização
def lematizacao (texto):
    lematizados = []
    for word in texto:
        tokens = nltk.word_tokenize(word)
        lematizado = ' '.join([lemmatizer.lemmatize(token) for token in tokens])
        lematizados.append(lematizado)
    return lematizados    

In [ ]:
# Criar teste de cada função

#Testa função limpa_texto
def limpa_texto_teste():
    
    # texto sem limpar
    texto_exemplo_sujo = ["Let's study Data Science e Analytics at Escola Superior de Agricultura 'Luiz de Queiros', which is in Piracicaba/SP - Brasil."]
    
    # texto apos limpeza
    texto_limpo = ["lets study data science e analytics at escola superior de agricultura luiz de queiros  which is in piracicaba sp  brasil"]

    # Faz a comparação
    for exemplo, resposta in zip(texto_exemplo_sujo, texto_limpo):

        # Retorna resultado da comparação
        if limpa_texto(exemplo) == resposta:
            return "Teste aprovado."
        else:
            raise Exception("Resposta não esperada em: '%s' " % exemplo)
    
# Executando a função de teste
try:
    limpa_texto_teste()
except NameError as e:
    print(e)
    

### Coleta de dados

In [ ]:
# Coleta dados
catalogoVulnerabilidades_bruto = pd.read_csv('https://www.cisa.gov/sites/default/files/csv/known_exploited_vulnerabilities.csv', header=0, sep=',')
#catalogoVulnerabilidades = pd.read_csv('dados/known_exploited_vulnerabilities.csv', header=0, sep=',')

# Mantém o DataFrame obtido originalmente sem alterações
catalogoVulnerabilidades = catalogoVulnerabilidades_bruto.copy()

# Criar coluna booleana state com 0 para Unknown e 1 para Known
catalogoVulnerabilidades['codigoKnownRansomwareCampaignUse'] = np.where(catalogoVulnerabilidades['knownRansomwareCampaignUse'] == 'Known', 1, 0 )

#Reordenando as colunas
catalogoVulnerabilidades = catalogoVulnerabilidades[['cveID',
                                                     'vendorProject',
                                                     'product',
                                                     'vulnerabilityName',
                                                     'dateAdded',
                                                     'shortDescription',
                                                     'requiredAction',
                                                     'dueDate',
                                                     'knownRansomwareCampaignUse',
                                                     'codigoKnownRansomwareCampaignUse',
                                                     'notes']]    



In [ ]:
# Verificar tipos de colunas
catalogoVulnerabilidades.info()

In [ ]:
# Excluir a coluna notes
catalogoVulnerabilidades.drop(columns=['cveID', 'notes'], inplace = True)

In [ ]:
# Verificar alguns dados
display(catalogoVulnerabilidades)

In [ ]:
# Transformar tipos de dados para datas
catalogoVulnerabilidades.dateAdded = pd.to_datetime(catalogoVulnerabilidades.dateAdded)
catalogoVulnerabilidades.dueDate = pd.to_datetime(catalogoVulnerabilidades.dueDate)

##Tranforma a coluna codigoKnownRansomwareCampaignUse para boolean
catalogoVulnerabilidades['codigoKnownRansomwareCampaignUse'] = catalogoVulnerabilidades['codigoKnownRansomwareCampaignUse'].astype('bool')

##Transforma colunas em string
catalogoVulnerabilidades['shortDescription'] = catalogoVulnerabilidades['shortDescription'].astype('string')

In [ ]:
# Verificar tipos de colunas após transformação
catalogoVulnerabilidades.info()

In [ ]:
# DataFrame com pontos e caracteres especiais
pd_frequencia_com_pontos = prepara_para_grafico(catalogoVulnerabilidades, 'shortDescription')

pd_frequencia_com_pontos.nlargest(columns = "frequencia", n = 10).sort_values(by=['frequencia'], ascending=False)

## Tratamento dos dados

In [ ]:
# Exibe gráfico de pareto contendo caracteres especiais e stop words
exibe_pareto(pd_frequencia_com_pontos, 'frequencia', 10, 'purple', 'Contendo caracteres especiais e stop words')

In [ ]:
# Limpa a coluna 
pd_catalogoVulnerabilidades_coluna_limpa = pd.DataFrame(catalogoVulnerabilidades)
pd_catalogoVulnerabilidades_coluna_limpa['shortDescription'] = catalogoVulnerabilidades['shortDescription'].apply(lambda x: limpa_texto(str(x)).lower())
pd_catalogoVulnerabilidades_coluna_limpa.shortDescription

In [ ]:
# Tokeniza DataFrame limpo
tokens_coluna_limpa = tokeniza_palavras_coluna(pd_catalogoVulnerabilidades_coluna_limpa, 'shortDescription')

In [ ]:
# Carrega lista de stop words do inglês e acrescenta algumas palavras
sw_vulnerabilidade_ciberneticas = ["could", "vulnerability", "would"] # Ver quais stop words adicionar
sw_en = list(set(stopwords.words('english')))
sw_en_plus = sw_en + sw_vulnerabilidade_ciberneticas



In [ ]:
# Remove as stop words
pd_catalogoVulnerabilidades_sem_stop_words = pd.DataFrame(pd_catalogoVulnerabilidades_coluna_limpa)
pd_catalogoVulnerabilidades_sem_stop_words['shortDescription'] = pd_catalogoVulnerabilidades_sem_stop_words['shortDescription'].apply(lambda x: remove_stop_words(x, sw_en_plus))
pd_catalogoVulnerabilidades_sem_stop_words.shortDescription

In [ ]:
# Cria DataFrame sem stop words
pd_frequencia_sem_stop_words = prepara_para_grafico(pd_catalogoVulnerabilidades_sem_stop_words, 'shortDescription')

In [ ]:
# Exibe gráfico de pareto sem caracteres especiais e sem stop words
exibe_pareto(pd_frequencia_sem_stop_words, 'frequencia', 10, 'blue', 'Sem stop words e caracteres especiais')

In [ ]:
# Aplicando Stemming
pd_catalogoVulnerabilidades_stemming = pd_catalogoVulnerabilidades_sem_stop_words.copy()
pd_catalogoVulnerabilidades_stemming['shortDescription'] = pd_catalogoVulnerabilidades_stemming['shortDescription'].apply(lambda x: stemming(str(x)))
pd_catalogoVulnerabilidades_stemming.shortDescription

In [ ]:
# Prepara para exibição de gráfico com stemming
pd_frequencia_com_stemming = prepara_para_grafico(pd_catalogoVulnerabilidades_stemming, 'shortDescription')

In [ ]:
# Gráfico de Pareto depois do stemming
exibe_pareto(pd_frequencia_com_stemming, 'frequencia', 10, 'yellow', 'Stemming')

In [ ]:
# Aplicando Lemmatization
pd_catalogoVulnerabilidades_lemmatization = pd_catalogoVulnerabilidades_sem_stop_words.copy()
pd_catalogoVulnerabilidades_lemmatization['shortDescription'] = lematizacao(pd_catalogoVulnerabilidades_lemmatization['shortDescription'])

pd_catalogoVulnerabilidades_lemmatization.shortDescription

In [ ]:
# Prepara para exibição de gráfico com lemmatization
df_frequencia_lemmatization = prepara_para_grafico(pd_catalogoVulnerabilidades_lemmatization, 'shortDescription')

In [ ]:
# Palavras que mais ocorreram após a lemmatização
df_frequencia_lemmatization.nlargest(columns = "frequencia", n = len(df_frequencia_lemmatization))

In [ ]:
# Palavras que menos ocorreram após a lemmatização
df_frequencia_lemmatization.nsmallest(columns = "frequencia", n = len(df_frequencia_lemmatization)).sort_values(by=['frequencia'])

In [ ]:
# Tokens com mais de 100 ocorrências

df_frequencia_lemmatization_qtde = df_frequencia_lemmatization[(df_frequencia_lemmatization.frequencia >= 100)].sort_values(by = ['frequencia'], ascending=False)

df_frequencia_lemmatization_qtde.head(15)

In [ ]:
# Gráfico de Pareto depois do Lemmatization
exibe_pareto(df_frequencia_lemmatization_qtde, 'frequencia', 10, 'green', 'Após lemmatization')

In [ ]:
# Vetorizar coluna e criar a bag of words (sacola de palavras)
#vetorizar = CountVectorizer(lowercase = False, max_features=2731)
vetorizar = CountVectorizer(lowercase = False, max_features=2300)
bow = vetorizar.fit_transform(pd_catalogoVulnerabilidades_lemmatization.shortDescription)

print(bow.shape)

In [ ]:
# Tranforma matriz esparsa em Dataframe
df_shortDescription_bow = pd.DataFrame.sparse.from_spmatrix(bow, columns=vetorizar.get_feature_names_out())

df_shortDescription_bow.head()

In [ ]:
# Wordcloud
shortDescription_all_words = ' '.join([word for word in pd_catalogoVulnerabilidades_lemmatization.shortDescription])

# Quantidade de palavras
print(len(shortDescription_all_words))

In [ ]:
# Exibindo bag of words
shortDescription_all_words

In [ ]:
# Gerar wordcloud

## https://amueller.github.io/word_cloud/generated/wordcloud.WordCloud.html

shortDescription_wc = WordCloud(width= 800, height= 500, max_font_size = 110, collocations=False).generate(shortDescription_all_words)

In [ ]:
# Plotar wordcloud
plt.figure(figsize=(10, 6))
plt.imshow(shortDescription_wc, interpolation='bilinear') #ver outras interpolações
plt.axis("off")
plt.show()

In [ ]:
# TF-IDF

## Exemplo. Analisar a aplicação correta
tfidf = TfidfVectorizer(lowercase = False, max_features=200)
regressao_logistica = LogisticRegression(solver = "lbfgs")

catalogoVulnerabilidades_tfidf_bruto = tfidf.fit_transform(pd_catalogoVulnerabilidades_lemmatization['shortDescription'])

catalogoVulnerabilidades_treino, catalogoVulnerabilidades_teste, catalogoVulnerabilidades_classe_treino, catalogoVulnerabilidades_classe_teste = train_test_split(catalogoVulnerabilidades_tfidf_bruto,
                                                                                                                                                                  pd_catalogoVulnerabilidades_lemmatization['knownRansomwareCampaignUse'], random_state = 42)

regressao_logistica.fit(catalogoVulnerabilidades_treino, catalogoVulnerabilidades_classe_treino)

acuracia_catalogoVulnerabilidades_tfidf_bruto = regressao_logistica.score(catalogoVulnerabilidades_teste, catalogoVulnerabilidades_classe_teste)

print(acuracia_catalogoVulnerabilidades_tfidf_bruto)

In [ ]:
# Ngrams

## Exemplo. Analisar a aplicação correta
tfidf = TfidfVectorizer(lowercase=False, ngram_range=(1,2))
catalogoVulnerabilidades_vetor_tfidf = tfidf.fit_transform(pd_catalogoVulnerabilidades_lemmatization['shortDescription'])

catalogoVulnerabilidades_vetor_tfidf_treino, catalogoVulnerabilidades_vetor_tfidf_teste, catalogoVulnerabilidades_vetor_tfidf_classe_treino, catalogoVulnerabilidades_vetor_tfidf_classe_teste = train_test_split(catalogoVulnerabilidades_vetor_tfidf,
                                                                                                                                                                  pd_catalogoVulnerabilidades_lemmatization['knownRansomwareCampaignUse'], random_state = 42)

regressao_logistica.fit(catalogoVulnerabilidades_vetor_tfidf_treino, catalogoVulnerabilidades_vetor_tfidf_classe_treino)

acuracia_catalogoVulnerabilidades_vetor_tfidf_ngrams = regressao_logistica.score(catalogoVulnerabilidades_vetor_tfidf_teste, catalogoVulnerabilidades_vetor_tfidf_classe_teste)

print(acuracia_catalogoVulnerabilidades_vetor_tfidf_ngrams)

In [ ]:
# Pesos de cada termo

## Exemplo. Analisar a aplicação correta
pesos_termo = pd.DataFrame(
    regressao_logistica.coef_[0].T,
    index = tfidf.get_feature_names_out()
)

pesos_termo.nlargest(12, 0)